In [1]:
#%cd ../../../
%cd /home/simonreif/cee-us
import os
import numpy as np
import pickle
import smart_settings
from mbrl.environments import env_from_string
#from mbrl.controllers.fb import ForwardBackwardController
# Needed for the recording only if LD_PRELOAD is not unset!
from mujoco_py import GlfwContext
GlfwContext(offscreen=True)
from mbrl.offline_helpers import buffer_utils as bu
from mbrl.offline_helpers.eval import calculate_success_rates
from mbrl.offline_helpers import misc


/home/simonreif/cee-us
Creating offscreen glfw


In [2]:
path_1 = "results/cee_us/construction/2blocks/gnn_ensemble_cee_us_freeplay/checkpoints_225/rollouts_wog"
path_flip_wog ="results/cee_us/zero_shot/2blocks/225iters/construction_flip/gnn_ensemble_icem/checkpoints_000/rollouts_wog"
path_flip_wg="results/cee_us/zero_shot/2blocks/225iters/construction_flip/gnn_ensemble_icem/checkpoints_000/rollouts"
path_comb = "datasets/construction/fb/freeplay_plus_flip/rollouts_wog"
path_trunc = "datasets/construction/bc/truncated/flip/rollouts_wog"


In [3]:
#fpf_buffer = bu.combine_buffers_from_dirs(path_1, path_2, path_comb)
#print("Buffer contains {} rollouts, each with length {}".format(len(fpf_buffer), fpf_buffer[0]["observations"].shape[0]))

In [4]:
# with open(path_1, 'rb') as f:
#      buffer_fp = pickle.load(f)
with open(path_flip_wog, "rb") as f:
    buffer_flip = pickle.load(f)
with open(path_flip_wg, "rb") as f:
    buffer_flip_wg = pickle.load(f)
with open(path_trunc, "rb") as f:
    buffer_trunc = pickle.load(f)
    
#print("Freeplay Buffer contains {} rollouts, each with length {}".format(len(buffer_fp), buffer_fp[0]["observations"].shape[0]))
print("Flip Buffer contains {} rollouts, each with length {}".format(len(buffer_flip), buffer_flip[0]["observations"].shape[0]))

Flip Buffer contains 1500 rollouts, each with length 100


In [5]:
working_dir = os.path.dirname(os.path.dirname(path_flip_wg))
params = smart_settings.load(os.path.join(working_dir, 'settings.json'), make_immutable=False)
env = env_from_string(params.env, **params["env_params"])


In [7]:
ep_id = 15
step_id = 1
goal_from_obs = env.goal_from_observation(buffer_flip_wg[ep_id]["next_observations"][step_id])
print(f"goal_from_obs: {goal_from_obs}")
goal_from_state = env.goal_from_state(buffer_flip[ep_id]["env_states"][step_id])
print(f"goal_from_state: {goal_from_state}")
obs_wog = buffer_flip[ep_id]["next_observations"][step_id]
print(f"obs_wog: {obs_wog}")
obs_recomb = env.append_goal_to_observation(obs_wog, goal_from_state)
print(f"obs_recomb: {obs_recomb}")
obs = buffer_flip_wg[ep_id]["next_observations"][step_id]
print(f"obs: {obs}")
obs-obs_recomb

goal_from_obs: [1.57079637 0.         0.         1.57079637 0.         0.
 0.         0.         0.        ]
goal_from_state: [1.57079633 0.         0.         1.57079633 0.         0.
 0.         0.         0.        ]
obs_wog: [ 1.36575913e+00  7.37307966e-01  5.48614264e-01  1.26626794e-04
  3.73157352e-04 -6.76568085e-03  8.57364619e-04  8.21815897e-03
 -2.72476231e-04  2.97866616e-04  1.45041215e+00  8.04565132e-01
  4.24882352e-01 -0.00000000e+00  0.00000000e+00 -0.00000000e+00
 -3.55528076e-20  1.83153830e-20  1.43005209e-05  2.31186249e-18
  4.88829414e-18 -5.92531445e-20  1.25925732e+00  6.43819630e-01
  4.24882680e-01 -0.00000000e+00  0.00000000e+00 -0.00000000e+00
 -1.43651368e-20 -1.30022687e-20  1.38340829e-05  2.73016793e-18
 -3.17791624e-18 -7.73549892e-19]
obs_recomb: [ 1.36575913e+00  7.37307966e-01  5.48614264e-01  1.26626794e-04
  3.73157352e-04 -6.76568085e-03  8.57364619e-04  8.21815897e-03
 -2.72476231e-04  2.97866616e-04  1.45041215e+00  8.04565132e-01
  4.248823

array([0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 4.37113901e-08, 0.00000000e+00,
       0.00000000e+00, 4.37113901e-08, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00])

In [ ]:
buffer_with_goals_path = os.path.join(working_dir, "checkpoints_000/rollouts")
with open(buffer_with_goals_path, "rb") as f:
    buffer_with_goals = pickle.load(f)
success_rates = calculate_success_rates(env, buffer_with_goals)
# rewards_comprew_eps0 = [env.compute_reward_from_obs_wg(obs) for obs in buffer_with_goals[0]["next_observations"]]
# rewards_comprew_eps0 = np.array(rewards_comprew_eps0)

In [9]:
idx = 1
s_rewards_zeroes, s_rewards_ones, s_rewards_twos = 0, 0, 0
for idx in range(43, 55):
    successes = np.array(env.eval_success(buffer_with_goals[idx]["next_observations"]))
    rewards = buffer_with_goals[idx]["rewards"]
    rewards_zeroes, rewards_ones, rewards_twos = misc.cr_es_agreement(rewards, successes)
    s_rewards_zeroes += rewards_zeroes
    s_rewards_ones += rewards_ones
    s_rewards_twos += rewards_twos
print(f"Sum of rewards: {s_rewards_zeroes}, {s_rewards_ones}, {s_rewards_twos}")

Mean rewards for successes 0, 0.5 and 1 are -2.0092587769031525, -2.0079834772192915, -2.008459070454473
Mean rewards for successes 0, 0.5 and 1 are -2.008941501379013, -1.9572747318368209, -2.0121837236263134
Mean rewards for successes 0, 0.5 and 1 are -2.009290647506714, -1.9958116718700953, -1.2065313160419464
Mean rewards for successes 0, 0.5 and 1 are -2.0091125263887295, -2.0099314321954567, 0
Mean rewards for successes 0, 0.5 and 1 are -2.009492286046346, -2.009712076187134, -2.013160715103149
Mean rewards for successes 0, 0.5 and 1 are -2.0088728533850775, -1.962670123854349, -1.008606421450774
Mean rewards for successes 0, 0.5 and 1 are -2.0092990928226047, -1.998055676837544, 0
Mean rewards for successes 0, 0.5 and 1 are -2.008280348777771, -1.0205655376116434, 0
Mean rewards for successes 0, 0.5 and 1 are -2.0090440011024473, 0, 0
Mean rewards for successes 0, 0.5 and 1 are -2.008790595190866, -1.0491652154922486, 1.0
Mean rewards for successes 0, 0.5 and 1 are -2.0089379284

In [ ]:
print(np.unique(successes, return_index=True, return_counts=True))


Mean rewards for successes 0, 0.5 and 1 are -2.0093651468103584, nan, -1.0082491874694823


/home/simonreif/anaconda3/envs/mbrl/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/simonreif/anaconda3/envs/mbrl/lib/python3.8/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [10]:
print(np.unique(successes, return_index=True, return_counts=True))

(array([0., 1., 2.], dtype=float32), array([ 0, 10, 31]), array([11, 20, 69]))


In [ ]:
for i in range(20):
    print(len(buffer_trunc[i]))

32
96
100
57
52
49
100
100
100
100
64
11
100
41
33
46
100
100
100
41


In [8]:
batch_1 = buffer_flip.sample(10)
batch_2 = buffer_trunc.sample(20)
print(batch_1.shape, batch_2.shape)
batch_comb = np.concatenate((batch_1, batch_2))
print(batch_comb.shape)
print(batch_comb["actions"].shape)

(10,) (20,)
(30,)
(30, 4)


In [11]:
env_states = batch_1["env_states"]
print(env_states.shape)

(10, 66)


In [ ]:
from mbrl.offline_helpers.buffer_utils import filter_buffer_by_length
filt_buffer = filter_buffer_by_length(buffer_trunc, 99)

In [10]:
len(filt_buffer)
for i in range(20):
    print(len(filt_buffer[i]))

32
96
57
52
49
64
11
41
33
46
41
32
78
47
35
59
71
69
32
47


In [18]:
np.unique(buffer_with_goals["successes"], return_counts=True)

(array([0., 1.]), array([123309,  26691]))

In [22]:
data = buffer_with_goals[0]._data
data_mod = data[:30]

In [25]:
data_mod["observations"].shape

(30, 43)

In [ ]:
buffer_with_goals[]

In [7]:
#msd_fp = mean_squared_diffs(buffer_fp)
msd_flip = mean_squared_diffs(buffer_flip)
#print(msd_fp)
print(msd_flip)

vars (34,)
0.3667941443926655


In [8]:
print(buffer_flip[0].dtype)

[('observations', 'f8', (34,)), ('next_observations', 'f8', (34,)), ('actions', 'f8', (4,)), ('rewards', 'f8', ()), ('dones', 'f8', ()), ('env_states', 'f8', (66,)), ('is_success', 'f8', ()), ('successes', 'f8', ())]


In [9]:
data = buffer_flip[0]._data

In [17]:
print(data["successes"])
print(data.keys())

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1.]


AttributeError: 'numpy.ndarray' object has no attribute 'keys'

In [ ]:
rew = buffer_flip["rewards"]
dons = buffer_flip["dones"]
is_succ = buffer_flip["is_success"]
succcesses = buffer_flip["successes"]

In [ ]:
print(sum(succcesses==1))
print(sum(is_succ))
print(sum(rew))
print(sum(dons))

26691
0.0
-264305.32527935505
0.0


In [ ]:
for i in range(8, 12):
    print(buffer_flip[i]["successes"])

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1.

In [ ]:
fpf_buffer = bu.combine_buffers_from_dirs(path_1, path_2, path_comb)
print("Buffer contains {} rollouts, each with length {}".format(len(fpf_buffer), fpf_buffer[0]["observations"].shape[0]))

: 

In [ ]:
import copy
new_customlist = copy.deepcopy(buffer_fp.rollouts)
new_customlist.extend(copy.deepcopy(buffer_flip.rollouts))


In [ ]:
from mbrl.rolloutbuffer import RolloutBuffer
new_buffer = RolloutBuffer(rollouts=new_customlist)

Flatten of RolloutBuffer with different fields, use only ['observations', 'rewards', 'successes', 'is_success', 'next_observations', 'env_states', 'actions', 'dones']


TypeError: Concatenating rollouts failed with error The DTypes <class 'numpy.dtype[float64]'> and <class 'numpy.dtype[void]'> do not have a common DType. For example they cannot be stored in a single array unless the dtype is `object`.

In [ ]:
for key in buffer_fp.common_field_names():
    print(buffer_fp[key].dtype)
print("___")
for key in buffer_flip.common_field_names():
    print(buffer_flip[key].dtype)

float64
float64
float64
float64
float64
float64
float64
float64
___
float64
float64
float64
float64
float64
float64
float64
float64


In [ ]:
def mean_squared_diffs(buffer, last_obs=True):
    if last_obs:
        obs=buffer.get_last_obs()
    else:
        raise NotImplementedError("Other obs selection not implemented.")
    vars = obs.var(axis=0)
    print(f"vars {vars.shape}")
    msd = 2*vars.mean()
    return msd

## Choose if to replay free play data

In [ ]:
print(list(buffer_fp.common_field_names()))
print(list(buffer_flip.common_field_names()))

['dones', 'next_observations', 'rewards', 'env_states', 'successes', 'is_success', 'actions', 'observations']
['dones', 'next_observations', 'rewards', 'env_states', 'successes', 'is_success', 'actions', 'observations']


In [ ]:
#working_dir = f'results/cee_us/{env_name}/2blocks/gnn_ensemble_cee_us_freeplay'
working_dir = f"results/cee_us/zero_shot/2blocks/225iters/construction_flip/gnn_ensemble_icem"
#working_dir = "results/cee_us/zero_shot/2blocks/225iters/construction_stack/gnn_ensemble_icem"
# Specify which training iteration of cee-us to replay
iteration = 73
mode = "free_play"
env_name = "construction" # construction or "playground"

## Read settings file and initialize the corresponding environment

In [ ]:
params = smart_settings.load(os.path.join(working_dir, 'settings.json'), make_immutable=False)
env = env_from_string(params.env, **params["env_params"])

In [ ]:
params["env_params"]["case"]

'Flip'

In [ ]:
env.nObj
# for key, value in env.__dict__.items():
#     print(key, " : ", value)
env.observation_space.shape

(43,)

## Load rollout buffer

In [ ]:
# with open(os.path.join(working_dir, 'checkpoints_225/rollouts'), 'rb') as f:
#     buffer = pickle.load(f)

with open(os.path.join(working_dir, 'checkpoints_000/rollouts'), 'rb') as f:
    buffer = pickle.load(f)

    
print("Buffer contains {} rollouts, each with length {}".format(len(buffer), buffer[0]["observations"].shape[0]))

Buffer contains 1500 rollouts, each with length 100


In [ ]:
buffer[0]["observations"].shape

(100, 43)

with open(os.path.join(working_dir, 'checkpoints_latest/rollouts'), 'rb') as f:
    buffer = pickle.load(f)
    
print("Buffer contains {} rollouts, each with length {}".format(len(buffer), buffer[0]["observations"].shape[0]))

In [ ]:
from mbrl.offline_helpers.buffer_utils import get_buffer_wo_goals
buffer_wog=get_buffer_wo_goals(buffer, env)

In [ ]:
mean, std = buffer_wog.get_mean_std()

In [ ]:
print(mean.mean(), std.mean())

0.26525406716904903 0.2028654744045809


In [ ]:
normed_obs=(buffer_wog["observations"]-mean)/std

In [ ]:
normed_obs.std()

0.9998977318394802

In [ ]:
batch = buffer_wog.sample(1024)
type(batch)

numpy.ndarray

In [ ]:
np.sum(buffer_wog["rewards"])
np.sum(buffer_wog["successes"])

40163.0

In [ ]:
next_obs = batch["next_observations"]
print(f"min {np.min(next_obs)}")
print(f"max {np.max(next_obs)}")
print(f"mean {np.mean(next_obs)}")
print(f"std {np.std(next_obs)}")

min -3.1415927410125732
max 3.1415927410125732
mean 0.2165381196116669
std 0.6853841297656907


In [ ]:
obs_g = buffer[0]["next_observations"]
obs_wog = buffer_wog[0]["next_observations"]
print(obs_g.shape)
print(obs_wog.shape)

(100, 43)
(100, 34)


In [ ]:
rollouts=buffer.rollouts
#print(rollouts[0]["observations"].shape)
#obs_wog=env.observation_wo_goal(rollouts[0]["observations"])
#print(obs_wog.shape)
rollout=rollouts[0]
print(rollout.field_names())


['observations', 'next_observations', 'actions', 'rewards', 'dones', 'env_states', 'is_success', 'successes']
['next_observations', 'env_states', 'dones', 'rewards', 'successes', 'observations', 'actions']


In [ ]:
from mbrl import torch_helpers

#obs, actions, next_obs = torch_helpers.to_tensor_device(buffer.get_random_transitions(120))
obs, actions, next_obs=buffer.get_random_transitions(7)
print(obs.shape, actions.shape, next_obs.shape)
print(type(obs), type(actions), type(next_obs))
obs, actions, next_obs = torch_helpers.to_tensor_device(obs, actions, next_obs)
print(type(obs), type(actions), type(next_obs))

(7, 43) (7, 4) (7, 43)
<class 'numpy.ndarray'> <class 'numpy.ndarray'> <class 'numpy.ndarray'>
<class 'torch.Tensor'> <class 'torch.Tensor'> <class 'torch.Tensor'>


In [ ]:
from mbrl import torch_helpers

obs, actions, next_obs = torch_helpers.to_tensor_device(*buffer.get_random_transitions(7))
print(type(obs), type(actions), type(next_obs))

<class 'torch.Tensor'> <class 'torch.Tensor'> <class 'torch.Tensor'>


In [ ]:
sampl = buffer.sample(133)
sampl["observations"].shape
sampl["actions"].shape

(133, 4)

In [ ]:
obs_array.shape[-1]

73

In [ ]:
num_inference_samples=50_000
print(obs_array.shape)
print(type(obs_array))
obs_flat=obs_array.reshape(-1, obs_array.shape[-1])
print(obs_flat.shape)
indices=np.random.choice(obs_flat.shape[0], num_inference_samples)
inf_obs=obs_flat[indices]
print(inf_obs.shape)
print(indices[110:130])

(1520, 100, 73)
<class 'numpy.ndarray'>
(152000, 73)
(50000, 73)
[ 31916 122113  17954   2706 100959   7607  96876 110021  97621 120342
  15700     86  86608   8702 137435  20870  92561 122898  84766  62423]


In [ ]:
i=2
print(type(flat_buffer[2][i]))
flat_buffer[2][i]

<class 'numpy.ndarray'>


array([ 0.69924605,  0.29749206, -1.        ,  0.71758622])

In [ ]:
import torch

In [ ]:
# for key in buffer[1]["actions"][1:10]:
#     print(key)
# rollout_id=1519
# print(buffer[rollout_id]["observations"].shape)
# print(any(abs(buffer[rollout_id]["observations"][0])>1))
# print(buffer[rollout_id]["observations"][0])
print(obs_array.shape)
obs_array=torch.tensor(obs_array)
c_obs_array=torch.flatten(obs_array, end_dim=1)
print(c_obs_array.shape)

torch.Size([1520, 100, 73])
torch.Size([152000, 73])


/tmp/ipykernel_16865/2563842894.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  obs_array=torch.tensor(obs_array)


## Record video

In [ ]:
video, video_path = setup_video(output_path, f'_{mode}_iter{iteration}', "2blocks_", env.get_fps())

ep_length = buffer[0]["observations"].shape[0]

render_width = 768
render_height = 512

ep_id = iteration * params.number_of_rollouts 
print(ep_id, ep_id+params.number_of_rollouts, len(buffer))
for i in range(ep_id, min(ep_id + params.number_of_rollouts, len(buffer))):
    obs = env.reset()
    for t in range(ep_length):
        if env.name == "PlaygroundwGoals":
            env.set_state_from_observation(buffer[i]["observations"][t, :])
        else:
            # Robotics environments require setting the whole env state for replay!
            env.set_GT_state(buffer[i]["env_states"][t, :])
        frame = env.render("rgb_array", render_width, render_height)
        video.append_data(frame)
        if env.name == "FetchPickAndPlaceConstruction":
            del env.viewer._markers[:]

video.close()

Record video in video/2blocks_rollout_free_play_iter100.mp4
2000 2020 2020


[libx264 @ 0x347d5a40] -qscale is ignored, -crf is recommended.


# Render rollouts with openglw

You need to set the following variable to render: 
```bash
export LD_PRELOAD=/usr/lib/x86_64-linux-gnu/libGLEW.so
```

In [ ]:
ep_id = iteration * params.number_of_rollouts 
success_rate = []
for i in range(ep_id, min(ep_id + params.number_of_rollouts, len(buffer))):
    episode_length = buffer[i]["observations"].shape[0]
    for t in range(episode_length):
        if env.name == "PlaygroundwGoals":
            env.set_state_from_observation(buffer[i]["observations"][t, :])
        else:
            # Robotics environments require setting the whole env state for replay!
            env.set_GT_state(buffer[i]["env_states"][t, :])
        env.render()    
env.close()

Creating window glfw


: 

In [ ]:
if mode != "free_play":
    success_rate = []
    for i in range(len(buffer)):
        rollout_success = env.eval_success(buffer[i]["next_observations"])
        stable_T = 5
        if env.name == "FetchPickAndPlaceConstruction" and "tower" in env.case:
            # Stack is only successful if we have a full tower! 
            # Check if the tower is stable for at least 5 timesteps
            dy = np.diff(rollout_success)
            success = np.logical_and(rollout_success[1:]==env.num_blocks, dy==0)
            success_rate.append(np.sum(success)>stable_T)
        elif env.name == "FetchPickAndPlaceConstruction" and env.case == 'PickAndPlace':
            # We determine success as highest number of solved elements with at least 5 timesteps of success
            u, c = np.unique(rollout_success, return_counts = True)
            # u: unique values, c: counts
            count_of_highest_success = c[np.argmax(u)]    
            success_rate.append(u[c>stable_T][-1]/env.nObj)
        else:
            # For flip, throw and Playground env push tasks: just get success at the end of rollout
            success_rate.append(rollout_success[-1]/env.nObj)
    print("Success rate over {} rollouts is {}".format(len(buffer), np.asarray(success_rate).mean()))